### Epirical Asset Pricing Part 2 - Boosted Regression Trees and Random Forests
Author: Ren Yang

In [ ]:
import pandas as pd
import scipy.stats as ss
import numpy as np
import datetime
import xgboost as xgb


from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [ ]:
# FOR COLAB
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/Students/'
file_name='openap_macro_merged.parquet.gzip'
stock_data=pd.read_parquet(data_dir+file_name)

In [ ]:
stock_sub=stock_data[stock_data['DateYM']>np.datetime64('2000-01-01')].reset_index()

### Data Cleaning and Preprocessing


In [ ]:
stock_sub_3=stock_sub.set_index(['permno','DateYM'],verify_integrity=True).drop('index', axis=1)
for i in stock_sub_3.columns[:-1]:
  if stock_sub_3[i].nunique()==2: # Check if column is binary
    stock_sub_3[i].fillna(0,inplace=True)
    stock_sub_3[i]=np.where(stock_sub_3[i]==0,-1,1) 
  else:
    stock_sub_3[i]=stock_sub_3[i].groupby('DateYM').apply(lambda x: x.fillna(0) if np.isnan(x.median()) else x.fillna(x.median()))# fill non binary column NaNs with period cross-sectional median. if period cross-sectional median donesn't exist, fill with 0                    
    stock_sub_3[i]=stock_sub_3[i].groupby('DateYM').rank(pct=True).transform(lambda x:2*((x-x.min())/(x.max()-x.min()))-1) 
                      

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/p

In [ ]:
df_rank=stock_sub_3[stock_sub_3.index.get_level_values(1)==pd.to_datetime('2020-12-31')] # rank based on the latest day
df_rank['mvel_rank']=df_rank['mvel1'].rank()
top_1000_permno=df_rank[df_rank['mvel_rank']>4746].index.get_level_values(0)
bot_1000_permno=df_rank[df_rank['mvel_rank']<=1000].index.get_level_values(0)


# slice top 1000 stocks
top_1000_df=stock_sub_3.loc[top_1000_permno, :]

# slice bot 1000 stocks
bot_1000_df=stock_sub_3.loc[bot_1000_permno,:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###Monthly Pediction on TOP 1000 Stocks using Random Forest Regressor

In [ ]:
# train for top 1000 stocks RF


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_top_1000_RF={}

ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_top_1000_RF=pd.DataFrame(columns=['ret_pred'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=top_1000_df.loc[top_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=top_1000_df.loc[(top_1000_df.index.get_level_values(1)>=start_of_test) & (top_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 

  RF = xgb.XGBRFRegressor(objective='reg:squarederror',learning_rate=1,max_depth=6,num_parallel_tree=300,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  RF.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')

  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  


  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
    
     count+=1
     
     y_pred=RF.predict(data=df.drop(columns=['retadj']))

     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     month_pred=pd.DataFrame(columns=['ret_pred'],index=df.index)

     month_pred['ret_pred']=y_pred

     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     cycle_prediction_results_top_1000_RF=pd.concat([cycle_prediction_results_top_1000_RF,month_pred])
  
  cycle_r_2_results_top_1000_RF[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)








Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


### Monthly Pediction on BOT 1000 Stocks using Random Forest Regressor

In [ ]:
# train for bot 1000 stocks RF


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_bot_1000_RF={}


ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_bot_1000_RF=pd.DataFrame(columns=['ret_pred'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=bot_1000_df.loc[bot_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=bot_1000_df.loc[(bot_1000_df.index.get_level_values(1)>=start_of_test) & (bot_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 

  RF = xgb.XGBRFRegressor(objective='reg:squarederror',learning_rate=1,max_depth=6,num_parallel_tree=300,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  RF.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  

  print(f'Cycle{cycle_counter} model trained')

  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
   
     count+=1
     
     y_pred=RF.predict(data=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     month_pred=pd.DataFrame(columns=['ret_pred'],index=df.index)

     month_pred['ret_pred']=y_pred

     cycle_prediction_results_bot_1000_RF=pd.concat([cycle_prediction_results_bot_1000_RF,month_pred])
  
  
  cycle_r_2_results_bot_1000_RF[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)



Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


### Monthly Pediction on TOP 1000 Stocks using Gradient Boosting Trees

In [ ]:
# train for top 1000 stocks Boosting 


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_top_1000_GBT={}

ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_top_1000_GBT=pd.DataFrame(columns=['ret_pred'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=top_1000_df.loc[top_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=top_1000_df.loc[(top_1000_df.index.get_level_values(1)>=start_of_test) & (top_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 

  GBRT = xgb.XGBRegressor(objective='reg:squarederror',learning_rate=0.01,max_depth=2,n_estimators=1000,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  GBRT.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')

  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  


  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
    
     count+=1
     
     y_pred=GBRT.predict(data=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)
     
     month_pred=pd.DataFrame(columns=['ret_pred'],index=df.index)

     month_pred['ret_pred']=y_pred

     cycle_prediction_results_top_1000_GBT=pd.concat([cycle_prediction_results_top_1000_GBT,month_pred])
  
  cycle_r_2_results_top_1000_GBT[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)


Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


### Monthly Pediction on BOT 1000 Stocks using Gradient Boosting Trees

In [ ]:
# train for bot 1000 stocks Gradient Boosting


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_bot_1000_GBT={}

ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_bot_1000_GBT=pd.DataFrame(columns=['ret_pred'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=bot_1000_df.loc[bot_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=bot_1000_df.loc[(bot_1000_df.index.get_level_values(1)>=start_of_test) & (bot_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 
  GBRT = xgb.XGBRegressor(objective='reg:squarederror',learning_rate=0.01,max_depth=2,n_estimators=1000,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  GBRT.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')


  


  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
   
     count+=1
     
     y_pred=GBRT.predict(data=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     month_pred=pd.DataFrame(columns=['ret_pred'],index=df.index)

     month_pred['ret_pred']=y_pred

     cycle_prediction_results_bot_1000_GBT=pd.concat([cycle_prediction_results_bot_1000_GBT,month_pred])
  
  
  cycle_r_2_results_bot_1000_GBT[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)

Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


### Prediction Result

In [ ]:
# TOP 1000 Stocks Monthly prediction using RF
cycle_prediction_results_top_1000_RF

,,ret_pred
permno,DateYM,
10104,2009-01-31,0.130043
10107,2009-01-31,0.126703
10138,2009-01-31,0.125271
10145,2009-01-31,0.142221
10200,2009-01-31,0.132450
...,...,...
93306,2019-12-31,0.027253
93312,2019-12-31,0.023661
93374,2019-12-31,0.022503


In [ ]:
# TOP 1000 Stocks avg monthly r_2 using RF
cycle_r_2_results_top_1000_RF

{'Cycle0Average Monthly R2:': -1.4843163627388531,
 'Cycle1Average Monthly R2:': -1.4253690906917384,
 'Cycle2Average Monthly R2:': -1.958164958457968,
 'Cycle3Average Monthly R2:': -0.39641376432830056,
 'Cycle4Average Monthly R2:': -0.3237915636911692,
 'Cycle5Average Monthly R2:': -0.3957258796764885,
 'Cycle6Average Monthly R2:': -0.4866956800780852,
 'Cycle7Average Monthly R2:': -0.39303657120338426,
 'Cycle8Average Monthly R2:': -0.04071510012342403,
 'Cycle9Average Monthly R2:': -0.73552727852288,
 'Cycle10Average Monthly R2:': -0.5111214395072058}

In [ ]:
# BOT 1000 Stocks Monthly prediction using RF
cycle_prediction_results_bot_1000_RF

,,ret_pred
permno,DateYM,
10044,2009-01-31,0.000367
10232,2009-01-31,-0.025973
10253,2009-01-31,0.020853
10463,2009-01-31,-0.020700
10514,2009-01-31,0.004328
...,...,...
93347,2019-12-31,0.007217
93350,2019-12-31,0.019543
93373,2019-12-31,0.007282


In [ ]:
# BOT 1000 Stocks avg monthly r_2 using RF
cycle_r_2_results_bot_1000_RF

{'Cycle0Average Monthly R2:': -0.15603784552527222,
 'Cycle1Average Monthly R2:': -0.3477491398867036,
 'Cycle2Average Monthly R2:': -0.2936827074587684,
 'Cycle3Average Monthly R2:': -0.07875500377288046,
 'Cycle4Average Monthly R2:': -0.036184197268945124,
 'Cycle5Average Monthly R2:': -0.06375490113236104,
 'Cycle6Average Monthly R2:': -0.03302720142370471,
 'Cycle7Average Monthly R2:': -0.07092469721661183,
 'Cycle8Average Monthly R2:': -0.05398016313732279,
 'Cycle9Average Monthly R2:': -0.13611338174187168,
 'Cycle10Average Monthly R2:': -0.050942322372888565}

In [ ]:
# TOP 1000 Stocks Monthly prediction using GBT
cycle_prediction_results_top_1000_GBT

,,ret_pred
permno,DateYM,
10104,2009-01-31,0.028923
10107,2009-01-31,0.034263
10138,2009-01-31,0.027757
10145,2009-01-31,0.036569
10200,2009-01-31,0.056788
...,...,...
93306,2019-12-31,0.031810
93312,2019-12-31,0.025311
93374,2019-12-31,0.027983


In [ ]:
# TOP 1000 Stocks avg monthly r_2 using GBT
cycle_r_2_results_top_1000_GBT

{'Cycle0Average Monthly R2:': -0.3912286846109234,
 'Cycle1Average Monthly R2:': -0.6722105561244405,
 'Cycle2Average Monthly R2:': -0.7732777046733901,
 'Cycle3Average Monthly R2:': -0.21244976326728673,
 'Cycle4Average Monthly R2:': -0.2159670811583029,
 'Cycle5Average Monthly R2:': -0.2352118406659066,
 'Cycle6Average Monthly R2:': -0.24052339983937965,
 'Cycle7Average Monthly R2:': -0.2220170134781747,
 'Cycle8Average Monthly R2:': -0.030162006585720325,
 'Cycle9Average Monthly R2:': -0.40079412381986623,
 'Cycle10Average Monthly R2:': -0.2839842740699156}

In [ ]:
# BOT 1000 Stocks Monthly prediction using GBT
cycle_prediction_results_bot_1000_GBT

,,ret_pred
permno,DateYM,
10044,2009-01-31,-0.052695
10232,2009-01-31,-0.056931
10253,2009-01-31,-0.051061
10463,2009-01-31,-0.068424
10514,2009-01-31,-0.052400
...,...,...
93347,2019-12-31,0.002001
93350,2019-12-31,0.015791
93373,2019-12-31,0.007274


In [ ]:
# BOT 1000 Stocks avg monthly r_2 using GBT
cycle_r_2_results_bot_1000_GBT

{'Cycle0Average Monthly R2:': -0.15874684042691353,
 'Cycle1Average Monthly R2:': -0.2173910955753006,
 'Cycle2Average Monthly R2:': -0.2745671375116178,
 'Cycle3Average Monthly R2:': -0.10049247961318207,
 'Cycle4Average Monthly R2:': -0.03200931587242383,
 'Cycle5Average Monthly R2:': -0.06429856214407269,
 'Cycle6Average Monthly R2:': -0.023199826901042536,
 'Cycle7Average Monthly R2:': -0.08041576784274852,
 'Cycle8Average Monthly R2:': -0.06246878550000046,
 'Cycle9Average Monthly R2:': -0.15034552352994324,
 'Cycle10Average Monthly R2:': -0.05015869235144383}

### Monthly Pediction on ALL Stocks using Gradient Boosting Trees

In [ ]:
# train for ALL stocks Gradient Boosting Tree


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_ALL_GBT={}

ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_ALL_GBT=pd.DataFrame(columns=['ret_pred_GBT'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=stock_sub_3.loc[stock_sub_3.index.get_level_values(1)<end_of_validation]
  
  cycle_test=stock_sub_3.loc[(stock_sub_3.index.get_level_values(1)>=start_of_test) & (stock_sub_3.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 
  GBRT = xgb.XGBRegressor(objective='reg:squarederror',learning_rate=0.01,max_depth=2,n_estimators=1000,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  GBRT.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')


  


  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
   
     count+=1
     
     y_pred=GBRT.predict(data=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     month_pred=pd.DataFrame(columns=['ret_pred_GBT'],index=df.index)

     month_pred['ret_pred_GBT']=y_pred

     cycle_prediction_results_ALL_GBT=pd.concat([cycle_prediction_results_ALL_GBT,month_pred])
  
  
  cycle_r_2_results_ALL_GBT[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)

Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


In [ ]:
# train for ALL stocks RF


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_ALL_RF={}

ind=pd.MultiIndex.from_tuples([], names=(u'permno',u'DateYM'))
cycle_prediction_results_ALL_RF=pd.DataFrame(columns=['ret_pred_RF'],index=ind)


while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=stock_sub_3.loc[stock_sub_3.index.get_level_values(1)<end_of_validation]
  
  cycle_test=stock_sub_3.loc[(stock_sub_3.index.get_level_values(1)>=start_of_test) & (stock_sub_3.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 
  RF = xgb.XGBRFRegressor(objective='reg:squarederror',learning_rate=1,max_depth=6,num_parallel_tree=300,colsample_bylevel=0.3,tree_method='gpu_hist')
  
  RF.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  

  print(f'Cycle{cycle_counter} model trained')


  


  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
   
     count+=1
     
     y_pred=RF.predict(data=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     month_pred=pd.DataFrame(columns=['ret_pred_RF'],index=df.index)

     month_pred['ret_pred_RF']=y_pred

     cycle_prediction_results_ALL_RF=pd.concat([cycle_prediction_results_ALL_RF,month_pred])
  
  
  cycle_r_2_results_ALL_RF[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)

Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


In [ ]:
cycle_prediction_results_ALL_RF

,,ret_pred_RF
permno,DateYM,
10001,2009-01-31,-0.012099
10002,2009-01-31,0.028562
10025,2009-01-31,0.045289
10026,2009-01-31,0.038522
10028,2009-01-31,-0.028876
...,...,...
93426,2019-12-31,0.012261
93427,2019-12-31,0.019366
93429,2019-12-31,0.012971


In [ ]:
cycle_prediction_results_ALL_GBT

,,ret_pred_GBT
permno,DateYM,
10001,2009-01-31,-0.023244
10002,2009-01-31,-0.024307
10025,2009-01-31,-0.021842
10026,2009-01-31,-0.016583
10028,2009-01-31,-0.028631
...,...,...
93426,2019-12-31,0.008277
93427,2019-12-31,0.025158
93429,2019-12-31,0.012494


### Portfolio Prediction

In [ ]:
data_dir_port = '/content/drive/MyDrive/Students/Portfolio/'

file_name_CMA_ret='CMA_ret.parquet.gzip'
file_name_CMA_wt='CMA_wt.parquet.gzip'

file_name_RMW_ret='RMW_ret.parquet.gzip'
file_name_RMW_wt='RMW_wt.parquet.gzip'

file_name_SP_ret='SP500_VW_ret.parquet.gzip'
file_name_SP_wt='SP500_VW_wt.parquet.gzip'

file_name_FF_ret='FF-3_ret.parquet.gzip'
file_name_FF_wt='FF-3_wt.parquet.gzip'

In [ ]:
FF_ret_data=pd.read_parquet(data_dir_port+file_name_SP_ret)

FF_wt_data=pd.read_parquet(data_dir_port+file_name_SP_wt)

In [ ]:
FF_ret_data

,jdate,group,ret,Group
0,1987-01-31,All,0.143541,SP500
1,1987-02-28,All,0.041642,SP500
2,1987-03-31,All,0.026284,SP500
3,1987-04-30,All,-0.008727,SP500
4,1987-05-31,All,0.008747,SP500
...,...,...,...,...
379,2018-08-31,All,0.034121,SP500
380,2018-09-30,All,0.006546,SP500
381,2018-10-31,All,-0.066588,SP500
382,2018-11-30,All,0.020427,SP500


In [ ]:
def Port_Pred_Comp(data_dir_port,file_name_wt,file_name_ret,Prediction,Model_Type,Portfolio_Type,Long,Short):






  
  FF_ret_data=pd.read_parquet(data_dir_port+file_name_ret)

  FF_wt_data=pd.read_parquet(data_dir_port+file_name_wt)




  FF_ret_data.rename(columns={'jdate':'DateYM'},inplace=True)
  
  FF_ret_data.set_index(['DateYM'],inplace=True)
  
  FF_wt_data.rename(columns={'jdate':'DateYM'},inplace=True)
  
  FF_wt_data.set_index(['permno','DateYM'],inplace=True)

  FF_wt_pred=FF_wt_data.merge(Prediction,how='inner',on=['permno','DateYM'],validate='one_to_one')

  FF_wt_pred['weighted_ret']=FF_wt_pred['port_wt']*FF_wt_pred['ret_pred'+f'_{Model_Type}']*661

  FF_wt_compare=FF_wt_pred.merge(FF_ret_data,how='inner',on=['port','DateYM'],validate='many_to_one')


  if Portfolio_Type==1:
    FF_pred_series=FF_wt_compare.groupby(['DateYM',FF_wt_compare['port'].str[1:]])['weighted_ret'].sum()
    FF_real=FF_wt_compare.groupby(['DateYM',FF_wt_compare['port'].str[1:]])['ret'].sum()

    FF_ret_pred=[]
    for i in FF_pred_series.index.get_level_values(0):
      diff=FF_pred_series[(i,Long)]-FF_pred_series[(i,Short)]
      FF_ret_pred.append(diff)

    FF_ret_real=[]
    for i in FF_real.index.get_level_values(0):

      diff=FF_real[(i,Long)]-FF_real[(i,Short)]
      FF_ret_real.append(diff)

  else:
    FF_pred_series=FF_wt_compare.groupby(['DateYM',FF_wt_compare['port'].str[:1]])['weighted_ret'].sum()
    FF_real=FF_wt_compare.groupby(['DateYM',FF_wt_compare['port'].str[:1]])['ret'].sum()

    FF_ret_pred=[]
    for i in FF_pred_series.index.get_level_values(0):
      diff=FF_pred_series[(i,Long)]-FF_pred_series[(i,Short)]
      FF_ret_pred.append(diff)

    FF_ret_real=[]
    for i in FF_real.index.get_level_values(0):
      diff=FF_real[(i,Long)]-FF_real[(i,Short)]
      FF_ret_real.append(diff)
  


  print(f'{Model_Type}: Portfolio Prediction R_2 Result:{metrics.r2_score(y_true=FF_ret_real, y_pred=FF_ret_pred)}')

In [ ]:
# RF SMB
Port_Pred_Comp(data_dir_port,file_name_FF_wt,file_name_FF_ret,cycle_prediction_results_ALL_RF,Model_Type='RF',Portfolio_Type=0,Long='S',Short='B')

RF: Portfolio Prediction R_2 Result:-0.06354967815932877


In [ ]:
# GBT SMB
Port_Pred_Comp(data_dir_port,file_name_FF_wt,file_name_FF_ret,cycle_prediction_results_ALL_GBT,Model_Type='GBT',Portfolio_Type=0,Long='S',Short='B')

GBT: Portfolio Prediction R_2 Result:-0.052810598429498645


In [ ]:
# RF HML
Port_Pred_Comp(data_dir_port,file_name_FF_wt,file_name_FF_ret,cycle_prediction_results_ALL_RF,Model_Type='RF',Portfolio_Type=1,Long='H',Short='L')

RF: Portfolio Prediction R_2 Result:-0.08203560881004579


In [ ]:
# GBT HML
Port_Pred_Comp(data_dir_port,file_name_FF_wt,file_name_FF_ret,cycle_prediction_results_ALL_GBT,Model_Type='GBT',Portfolio_Type=1,Long='H',Short='L')

GBT: Portfolio Prediction R_2 Result:-0.11634187267832763


In [ ]:
# RF CMA
Port_Pred_Comp(data_dir_port,file_name_CMA_wt,file_name_CMA_ret,cycle_prediction_results_ALL_RF,Model_Type='RF',Portfolio_Type=1,Long='M',Short='A')

RF: Portfolio Prediction R_2 Result:-0.056520407911284876


In [ ]:
# GBT CMA
Port_Pred_Comp(data_dir_port,file_name_CMA_wt,file_name_CMA_ret,cycle_prediction_results_ALL_GBT,Model_Type='GBT',Portfolio_Type=1,Long='M',Short='A')

GBT: Portfolio Prediction R_2 Result:-0.03838701618110729


In [ ]:
# RF RMW
Port_Pred_Comp(data_dir_port,file_name_RMW_wt,file_name_RMW_ret,cycle_prediction_results_ALL_RF,Model_Type='RF',Portfolio_Type=1,Long='R',Short='W')

RF: Portfolio Prediction R_2 Result:-0.09837339535093514


In [ ]:
# GBT RMW
Port_Pred_Comp(data_dir_port,file_name_RMW_wt,file_name_RMW_ret,cycle_prediction_results_ALL_GBT,Model_Type='GBT',Portfolio_Type=1,Long='R',Short='W')

GBT: Portfolio Prediction R_2 Result:-0.09481329298852592


In [ ]:
# SP500 VW 

SP_ret_data=pd.read_parquet(data_dir_port+file_name_SP_ret)

SP_wt_data=pd.read_parquet(data_dir_port+file_name_SP_wt)


SP_ret_data.rename(columns={'jdate':'DateYM'},inplace=True)

SP_ret_data.set_index(['DateYM'],inplace=True)
  
SP_wt_data.rename(columns={'jdate':'DateYM'},inplace=True)
  
SP_wt_data.set_index(['permno','DateYM'],inplace=True)


In [ ]:
# Random Forest R_2

SP_wt_pred_RF=SP_wt_data.merge(cycle_prediction_results_ALL_RF,how='inner',on=['permno','DateYM'])

SP_wt_pred_RF['weighted_ret']=SP_wt_pred_RF['port_wt']*SP_wt_pred_RF['ret_pred_RF']

SP_wt_compare_RF=SP_wt_pred_RF.merge(SP_ret_data,how='inner',on=['DateYM'],validate='many_to_one')

metrics.r2_score(y_true=SP_wt_compare_RF['ret'], y_pred=SP_wt_compare_RF['weighted_ret'])

-0.08097717950895511

In [ ]:
# Gradient Boosting Tree R-2

SP_wt_pred_GBT=SP_wt_data.merge(cycle_prediction_results_ALL_GBT,how='inner',on=['permno','DateYM'])

SP_wt_pred_GBT['weighted_ret']=SP_wt_pred_GBT['port_wt']*SP_wt_pred_GBT['ret_pred_GBT']

SP_wt_compare_GBT=SP_wt_pred_GBT.merge(SP_ret_data,how='inner',on=['DateYM'],validate='many_to_one')

metrics.r2_score(y_true=SP_wt_compare_GBT['ret'], y_pred=SP_wt_compare_GBT['weighted_ret'])



-0.08127715253692469